# Train Transformer for the Motion2lang task

## Get sources

In [ ]:
// for colab
%install-location $cwd/swift-install
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/wojtekcz/language2motion.git", .branch("master"))' Datasets TranslationModels TextModels ModelSupport SummaryWriter MotionLangModels TrainingLoop

## What's the GPU?

In [ ]:
import Foundation

func shell(_ command: String) -> String {
    let task = Process()
    let pipe = Pipe()

    task.standardOutput = pipe
    task.arguments = ["-c", command]
    task.launchPath = "/bin/bash"
    task.launch()

    let data = pipe.fileHandleForReading.readDataToEndOfFile()
    return String(data: data, encoding: .utf8)!
}

func sh(_ command: String) {
    print(shell(command))
}

sh("""
export PATH="$PATH:/opt/bin:/swift/toolchain/usr/bin"
export LD_LIBRARY_PATH="/usr/lib64-nvidia:$LD_LIBRARY_PATH"
nvidia-smi
""")

## run colab ssh server

In [ ]:
// run colab ssh server
// after it finishes, interrupt cell execution
sh("bash <(curl -s https://raw.githubusercontent.com/wojtekcz/language2motion/master/notebooks/Colab/swift_colab_ssh_server.sh)")

In [ ]:
sh("ps ax|grep ssh")

In [ ]:
sh("kill -9 2211")  // enter ssh pid to kill the tunnel

## Imports

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import ModelSupport
import Datasets
import SummaryWriter
import MotionLangModels
import TrainingLoop
import x10_optimizers_optimizer

In [ ]:
import PythonKit

%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

## Download data

In [ ]:
// let dataset_name = "motion_dataset_v3.10Hz.multi"
let dataset_name = "motion_dataset_v3.10Hz.multi.mini"

In [ ]:
sh("mkdir -p /content/data/")
sh("""
cd /content/data/
wget -nv --show-progress -N https://github.com/wojtekcz/language2motion/releases/download/v0.3.0/\(dataset_name).tgz
wget -nv -N https://github.com/wojtekcz/language2motion/releases/download/v0.1.0/vocab.txt
tar xzvf \(dataset_name).tgz --skip-old-files
""")

## Set training params

In [ ]:
let runName = "run_11"
let batchSize = 100
let maxMotionLength = 50
let maxTextSequenceLength = 40
let nEpochs = 10

var optimizerOpts = OptimizerOpts(
    peakLearningRate: 5e-4,
    beta1: 0.9,
    beta2: 0.999,
    useBiasCorrection: false,
    lrSlopeMultiplier: 2,
    nEpochs: nEpochs
)

// let datasetSize: DatasetSize = .multi_full
let datasetSize: DatasetSize = .multi_mini

print("runName: \(runName)")
print("batchSize: \(batchSize)")
print("maxMotionLength: \(maxMotionLength)")
print("maxTextSequenceLength: \(maxTextSequenceLength)")
print("nEpochs: \(nEpochs)")
print("peakLearningRate: \(optimizerOpts.peakLearningRate)")
print("datasetSize: \(datasetSize)")
print("stepsPerEpoch: \(optimizerOpts.stepsPerEpoch)")

let dataURL = URL(fileURLWithPath: "/content/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")

let logdirURL = dataURL.appendingPathComponent("runs/Motion2lang/", isDirectory: true)
let rundirURL = logdirURL.appendingPathComponent(runName, isDirectory: true)
let checkpointURL = rundirURL.appendingPathComponent("checkpoints", isDirectory: true)

## Select eager or X10 backend

In [ ]:
let device = Device.defaultXLA
// let device = Device.defaultTFEager
print("backend: \(device)")

## X10 warm-up

In [ ]:
let eagerTensor1 = Tensor([0.0, 1.0, 2.0])
let eagerTensor2 = Tensor([1.5, 2.5, 3.5])
let eagerTensorSum = eagerTensor1 + eagerTensor2
// print(eagerTensorSum)
// print(eagerTensor1.device)
let x10Tensor2 = Tensor([1.5, 2.5, 3.5], on: Device.defaultXLA)
// print(x10Tensor2.device)

## instantiate text processor

In [ ]:
// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = LegacyTextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

## Load dataset

In [ ]:
print("\nLoading dataset...")

var dataset = try Motion2Lang(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    minMotionLength: 20,
    maxMotionLength: 100,
    trainTestSplit: 0.9,
    device: device
) { (motionSample: MotionSample) -> MotionLangBatch in    
    let singleBatch = textProcessor.preprocess(motionSample: motionSample, maxMotionLength: maxMotionLength, maxTextSequenceLength: maxTextSequenceLength)
    return singleBatch
}

print("Dataset acquired.")

## Instantiate model

In [ ]:
// instantiate model
let modelSize = 128
let config = MotionLangTransformerConfig(
    vocabSize: vocabulary.count,
    nbJoints: 47,
    layerCount: 6,
    modelSize: modelSize,
    feedForwardSize: 512,
    headCount: 4,
    dropoutProbability: 0.1,
    sentenceMaxPositionalLength: 100,
    motionMaxPositionalLength: 500
)

var start_epoch = 0

/// create new model
var model = MotionLangTransformer(config: config)

/// load model checkpoint
//print("logdirURL: \(logdirURL.path)")
// start_epoch = 17
// let modeName = "model.e\(start_epoch)"
//let modeName = "model.final"
// var model = try! MotionLangTransformer(checkpoint: logdirURL.appendingPathComponent("run_9/checkpoints"), config: config, name: modeName)

## Test model with one batch

In [ ]:
// // get a batch
// print("\nOne batch (MotionLangBatch):")
// var epochIterator = dataset.trainEpochs.enumerated().makeIterator()
// let epoch = epochIterator.next()
// let batches = Array(epoch!.1)
// let batch: MotionLangBatch = batches[0]
// print("type: \(type(of:batch))")
// print("motion.shape: \(batch.motion.shape)")
// // print("motionFlag.shape: \(batch.motionFlag.shape)")
// print("mask.shape: \(batch.mask.shape)")
// print("origMotionFramesCount.shape: \(batch.origMotionFramesCount.shape)")
// print("origMotionFramesCount: \(batch.origMotionFramesCount)")
// print("targetTokenIds.shape: \(batch.targetTokenIds.shape)")
// print("targetMask.shape: \(batch.targetMask.shape)")
// print("targetTruth.shape: \(batch.targetTruth.shape)")

In [ ]:
// // run one batch
// print("\nRun one batch:")
// print("==============")
// let deviceBatch = MotionLangBatch(copying: batch, to: device)
// let output = model(deviceBatch)
// print("output.shape: \(output.shape)")

## Optimizer

In [ ]:
optimizerOpts.stepsPerEpoch = dataset.motionSamples.count/batchSize // function of training set size and batching configuration
let optimizerWrapper = OptimizerWrapper(opts: optimizerOpts, model: model)

## Training helpers

In [ ]:
/// stats recorder
let statsRecorder = StatsRecorder(logdirURL: rundirURL)

In [ ]:
@differentiable(wrt: y_pred)
func embeddedSoftmaxCrossEntropy(y_pred: Tensor<Float>, y_true: MotionLangBatch.MLTarget) -> Tensor<Float> {
    let resultSize = y_true.targetTruth.shape.last! * y_true.targetTruth.shape.first!
    let logits = y_pred.reshaped(to: [resultSize, -1])
    let labels = y_true.targetTruth.reshaped(to: [-1])
    // TODO: ignore padded entries
    return softmaxCrossEntropy(logits: logits, labels: labels)
}

## Set up decoding

In [ ]:
func greedyDecodeSample(_ sample_id: Int, maxLength: Int = 15) {
    let motionSample = dataset.motionSampleDict[sample_id]!
    print("\nsample: \(motionSample.sampleID), \"\(motionSample.annotations[0])\", motion: \(motionSample.timesteps[-1]) sec (\(motionSample.motion.shape[0]) frames)")

    let singleBatch = textProcessor.preprocess(motionSample: motionSample, maxMotionLength: maxMotionLength, maxTextSequenceLength: maxTextSequenceLength)
    let out = MotionLangDecoder.greedyDecode(model: model, input: singleBatch.source, maxLength: maxLength, startSymbol: textProcessor.bosId)
    let outputStr = textProcessor.decode(tensor: out)
    print("decoded: \"\(outputStr)\"")
}

In [ ]:
let samplesToDecode = [
    ["sampleID": dataset.motionSamples[0].sampleID, "text": dataset.motionSamples[0].annotations[0]], // for small dataset
//    ["sampleID": 733, "text": "Ala ma kota."], // for .micro dataset
//    ["sampleID": 1242, "text": "Ala ma kota."], // for .multi_mini dataset   
    // ["sampleID": 449, "text": "A person runs forward."],
    // ["sampleID": 3921, "text": "A human is swimming."],
    // ["sampleID": 843, "text": "A person walks."],
    // ["sampleID": 1426, "text": "A person plays the air guitar."],
    // ["sampleID": 1292, "text": "A person performs a squat."],
    // ["sampleID": 1315, "text": "A human raises their left foot and touches it with the right hand."]
]

## Training loop

In [ ]:
// start tensorboard
// cd /content/data
// tensorboard --bind_all --logdir runs/Motion2lang/

In [ ]:
// let nEpochs = 10

In [ ]:
// Training loop
print("\nSetting up the training loop")
let trainingProgress = TrainingProgress(metrics: [.loss])
var trainingLoop: TrainingLoop = TrainingLoop(
    training: dataset.trainEpochs,
    validation: dataset.testBatches,
    optimizer: optimizerWrapper.optimizer,
    lossFunction:  embeddedSoftmaxCrossEntropy,
    callbacks: [trainingProgress.update, statsRecorder.writeStats, optimizerWrapper.learningRateUpdater]
)

print("\nTraining Transformer for the Motion2lang task!")
// FIXME: epoch loop workaround for checkpoint saving
for epochIndex in start_epoch..<start_epoch+nEpochs {
    print("epoch \(epochIndex+1)/\(start_epoch + nEpochs)")
    statsRecorder.epochIndex = epochIndex
    try! trainingLoop.fit(&model, epochs: 1, on: device)
    try! model.writeCheckpoint(to: checkpointURL, name: "model.e\(epochIndex+1)")

    Context.local.learningPhase = .inference
    model.move(to: Device.defaultTFEager)
    for sample in samplesToDecode {
        greedyDecodeSample(sample["sampleID"] as! Int, maxLength: 20)
    }
    model.move(to: device)
}

try! model.writeCheckpoint(to: checkpointURL, name: "model.final")
print("\nFinished training.")

## Generate motion description

In [ ]:
dataset.motionSamples[0].sampleID

In [ ]:
// let sample_id = 2410
// let sample_id = 446
// let sample_id = 449
// let sample_id = 733
let sample_id = 1242
Context.local.learningPhase = .inference
model.move(to: Device.defaultTFEager)
greedyDecodeSample(sample_id, maxLength: 100)
model.move(to: device)